In [1]:
import pytz
import aiohttp
import asyncio
import datetime


async def datetime_to_iso_string(date: datetime.datetime) -> str :
    return (
        date
        .replace(microsecond=0)
        .astimezone(pytz.timezone('Asia/Seoul'))
        .isoformat()
    )


async def request_sampling_data(size_per_channel: int, day_interval: int):
    url = 'http://k8s.mysterico.com:31464/analyzer/niz_sample_data/generate'
    async with aiohttp.ClientSession() as session:
        today = datetime.datetime.now()
        iso_today = await datetime_to_iso_string(today)
        interval = datetime.timedelta(days=day_interval)
        response = await session.post(
            url=url,
            json={
                "size_per_channel": size_per_channel,
                "target_keyword": ["삼쩜삼"],
                "start_date": await datetime_to_iso_string(today - interval),
                "end_date": iso_today,
                # "channel": ["twitter"],
                # "search_keyword": ["산타클로스"],
                "include": ["channel"]
            }
        )
        return await response.json()


async def random_sampling(size_per_channel: int, day_interval: int):
    response = await request_sampling_data(size_per_channel, day_interval)
    documents = response.get("documents")
    documents = [document.get("contentPlainText") for document in documents]
    return documents




In [2]:
documents = await random_sampling(
    size_per_channel=1000,
    day_interval=2 ** 10
)
len(documents)

6102

In [3]:
import unicodedata
from typing import List

async def request_ner(sentences: List[str]):
    url = 'http://127.0.0.1:8000/predict'
    sentences = [unicodedata.normalize('NFC', sentence) for sentence in sentences]
    async with aiohttp.ClientSession() as session:
        response = await session.post(
            url=url,
            json={
                "sentences": sentences
            }
        )
        return await response.json()

In [4]:
await request_ner(['🐛 Update : 쿠버플로우 auth 버그 캐치를 위한 커스텀 스크립트 추가'])

[{'sentence': '🐛 Update : 쿠버플로우 auth 버그 캐치를 위한 커스텀 스크립트 추가',
  'entities': [['쿠버플로우', 'PS'],
   ['auth', 'PS'],
   ['커스텀', 'PS'],
   ['스크립트', 'PS']]}]

In [5]:
import random

random.choice(documents)

"동생이 경양식 돈까쓰 땡긴다고 퇴근하고 같이 먹으러 갔다 \u200b 나빼고 다(가족들) 가본 돈까스브로스 웃긴건 난 한번도 안 가봤는데 왜 다 내 번호로 적립하는데 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 가끔씩 적립되었다고 카톡이 온다 (누군데...누가먹었는데.........?...) \u200b 드디어 나도 왔다 돈까스브로스 철산점 경기도 광명시 철산로 15 2-2호 스프가 나와주구요 동생이 떠다준 궁물 후루룩 다 먹으니까 또 채워주신 숲 경양식 돈까스만의 매력이잖아요 브로스돈까스와 매콤 돈까스 진짜 맛있었다.....조합인정 배 안 찰것 같았는데 다 먹고 배 엄청 불렀다는거^^...; \u200b 너무 배불러서 그렇게 쇼핑을 했습니다 JAJU도 들리고 올리브영도 들리고 (잠시후 올영에서 뭐샀는지 공개) 점심 안 먹으려고 집에 있던 호빵을 챙겨왔다 모짜렐라치즈피자 먹어보셨어요? \u200b 이거 진짜 대박 맛있어요 저의 세젤 호빵이 될 듯 합니다 몇 개 사서 쟁여두ㅓ야지 \u200b 퇴근하고 집가는데 닭볶음탕이 너무 먹고싶었다 배민 넘 많이 시켜먹어서 오랜만에 요리를 해먹자는 다짐을 ㅋㅋㅋㅋㅋㅋㅋ🧡 절단육+버섯+감자=8500원의 행복 \u200b 젤 자신있는 요리 닭볶음탕을 만들어보았습니다 보글보글 버섯 넘 귀요미들인데 버섯전골 같은 비쥬얼 파 고추 송송 썰어 졸여주면 됩니다 (궁물이 없어질때까지) \u200b 유튭을 틀어놓고 밥을 먹겠습니다 요즘 세상 젤 재밌는 곽튜브 곽준빈님 볼때마다 피식.......피쉭... 매콤한 닭볶음탕에 치즈이불을 덮고 와인이랑 먹습니다 이것이 진정한 꼬꼬뱅 아닌가요 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 너모 맛있게 잘 되서 두그릇이나 먹었답니다 \u200b 오랜만에 요리하니까 더 즐겁습니다 나름 거금을 들여산 엡손PM401포프는 간간히 잘 쓰이고 있다 얼마전에 만든 일러스트 작업물이 선명하게 잘 나와준다 이웃 블로그보다가 1월에 삼쩜삼 해서 환급받았는데 설마 또 나오겠어? 했다가 정말로 나왔다 수수료 4000원 내고 바로 환급금 신청 \u200b 애드포스트도 이젠 환

In [27]:
import random

sampling_trial = 10

random_documents = [random.choice(documents) for _ in range(sampling_trial)]
for random_document in random_documents:
    try:
        result = await request_ner([random_document])
        for k in result[0]:
            print(f'<<{k}>>')
            print(result[0].get(k), '\n')
        print('='*100, '\n')
    except Exception as e:
        print('\n', e)
        print(random_document, '\n')
        continue

<<sentence>>
삼쩜삼 앱으로 세금 환급금도 받고, 이벤 참여로 굿즈도 받았다💫👏💫 완전 청량! #삼쩜삼 #삼쩜삼굿즈 #삼쩜삼시티즌룩 #굿즈맛집 

<<merged_sentence>>
[삼쩜삼 : PS] 앱으로 세금 환급금도 받고, 이벤 참여로 [굿즈 : PS]도 받았다💫👏💫 완전 청량! #[삼쩜삼 : PS] #[삼쩜삼굿즈 : PS] #[삼쩜삼시티즌룩 : PS] #[굿즈 : PS]맛집 

<<entities>>
[['삼쩜삼', 'PS'], ['굿즈', 'PS'], ['삼쩜삼', 'PS'], ['삼쩜삼굿즈', 'PS'], ['삼쩜삼시티즌룩', 'PS'], ['굿즈', 'PS']] 


<<sentence>>
근로장려금, 부가가치세, 종합소득세 환급금 등 찾아가지 않은 국세 환급금이 5월 현재 1434억원에 달하는 것으로 나타났습니다.

'삼쩜삼'온라인 서비스도 이용할 수 있습니다. 참고하셔서 확인해보세요.. 

<<merged_sentence>>
근로장려금, 부가가치세, 종합소득세 환급금 등 찾아가지 않은 국세 환급금이 [5월 현재 : DT] 1434억원에 달하는 것으로 나타났습니다.  '[삼쩜삼 : PS]'온라인 서비스도 이용할 수 있습니다. 참고하셔서 확인해보세요.. 

<<entities>>
[['5월', 'DT'], ['현재', 'DT'], ['삼쩜삼', 'PS']] 


<<sentence>>
이번 부가세 신고 기간에 사장님 두 분이 삼쩜삼 조회 하셔서 홈택스 자료 다운을 못했었다
가뜩이나 시간도 부족한데 수임동의를 하고 있는 그 상황이 조금 짱났달까.. 

<<merged_sentence>>
[이번 : DT] 부가세 신고 기간에 사장님 두 분이 [삼쩜삼 : PS] 조회 하셔서 [홈택스 : PS] 자료 다운을 못했었다 가뜩이나 시간도 부족한데 수임동의를 하고 있는 그 상황이 조금 짱났달까.. 

<<entities>>
[['이번', 'DT'], ['삼쩜삼', 'PS'], ['홈택스', 'PS']] 


0, message='Attempt to d